In [1]:
import torch
import faiss
import numpy as np
from torchvision import datasets, transforms
from transformers import AutoImageProcessor, AutoModel
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from scipy.spatial.distance import chebyshev, minkowski
from tqdm import tqdm
from torch.utils.data import DataLoader
from collections import defaultdict
import pandas as pd
from torchvision import models as tv_models

models = {
    "ViT": "nateraw/vit-base-beans",
    "Swin": "microsoft/swin-base-patch4-window7-224",
    "ConvNeXT": "facebook/convnext-base-224",
    "RegNet": "facebook/regnet-y-040",
    "ResNet50": tv_models.resnet50,
    "ResNet101": tv_models.resnet101,
    "ResNet152": tv_models.resnet152,
    "DEiT": "facebook/deit-base-patch16-224",
}

def load_model_and_processor(model_name):
    if model_name in ["ResNet50", "ResNet101", "ResNet152"]:
        model = models[model_name](pretrained=True)
        model = torch.nn.Sequential(*(list(model.children())[:-1]))
        processor = None
    else:
        processor = AutoImageProcessor.from_pretrained(models[model_name])
        model = AutoModel.from_pretrained(models[model_name])
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    return model, processor

def get_image_dataset(directory):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset = datasets.ImageFolder(root=directory, transform=transform)
    return dataset

def get_embeddings(model, data_loader, device):
    embeddings = []
    labels = []
    for images, target in tqdm(data_loader):
        inputs = images.to(device)
        with torch.no_grad():
            outputs = model(inputs)
        if isinstance(outputs, torch.Tensor):
            embedding = outputs.squeeze().cpu().numpy()
        else:
            embedding = outputs.pooler_output.cpu().numpy() if hasattr(outputs, "pooler_output") else outputs.last_hidden_state[:, 0].cpu().numpy()
        embeddings.append(embedding.reshape(len(images), -1))
        labels.extend(target.numpy())
    return np.vstack(embeddings), labels

def calculate_accuracy(distances, train_labels, test_labels, k=1):
    indices = np.argsort(distances, axis=1)[:, :k]
    correct = np.any([np.array([train_labels[idx] for idx in indices[:, i]]) == test_labels for i in range(k)], axis=0)
    return np.mean(correct) * 100

def calculate_top_k_accuracy(distances, train_labels, test_labels, k):
    indices = np.argsort(distances, axis=1)[:, :k]
    correct_top_k = np.any([np.array([train_labels[idx] for idx in indices[:, i]]) == test_labels for i in range(k)], axis=0)
    return np.mean(correct_top_k) * 100

def calculate_person_wise_accuracy(distances, train_labels, test_labels, class_names, k=1):
    indices = np.argsort(distances, axis=1)[:, :k]
    correct = np.any([np.array([train_labels[idx] for idx in indices[:, i]]) == test_labels for i in range(k)], axis=0)
    person_accuracy = defaultdict(lambda: {"correct": 0, "total": 0})
    
    for i, label in enumerate(test_labels):
        person_accuracy[class_names[label]]["total"] += 1
        if correct[i]:
            person_accuracy[class_names[label]]["correct"] += 1
    
    return person_accuracy

def format_accuracy_table(person_accuracy):
    data = {
        "Person": [],
        "Correct Predictions": [],
        "Total Predictions": [],
        "Accuracy (%)": []
    }
    
    for person, stats in person_accuracy.items():
        accuracy = (stats["correct"] / stats["total"]) * 100
        data["Person"].append(person)
        data["Correct Predictions"].append(stats["correct"])
        data["Total Predictions"].append(stats["total"])
        data["Accuracy (%)"].append(f"{accuracy:.2f}")
    
    df = pd.DataFrame(data)
    return df

def evaluate_distance_metrics(model_name, train_embeddings, test_embeddings, train_labels, test_labels, class_names, results):
    l2_distances = euclidean_distances(test_embeddings, train_embeddings)
    top_1_acc = calculate_accuracy(l2_distances, train_labels, test_labels, k=1)
    top_3_acc = calculate_accuracy(l2_distances, train_labels, test_labels, k=3)
    top_k_acc = calculate_top_k_accuracy(l2_distances, train_labels, test_labels, k=5)

    results.append({
        "Model": model_name,
        "Top-1 Accuracy (%)": top_1_acc,
        "Top-3 Accuracy (%)": top_3_acc,
        "Top-5 Accuracy (%)": top_k_acc
    })

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_dataset = get_image_dataset("DATASET-1/train")
    test_dataset = get_image_dataset("DATASET-1/test")
    class_names = train_dataset.classes

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    results = []

    for model_name in models.keys():
        print(f"Evaluating model: {model_name}")
        model, processor = load_model_and_processor(model_name)
        
        train_embeddings, train_labels = get_embeddings(model, train_loader, device)
        test_embeddings, test_labels = get_embeddings(model, test_loader, device)
        
        evaluate_distance_metrics(model_name, train_embeddings, test_embeddings, train_labels, test_labels, class_names, results)

    results_df = pd.DataFrame(results)
    print("All Models Evaluation Results:")
    print(results_df)

if __name__ == "__main__":
    main()


Evaluating model: ViT


Some weights of ViTModel were not initialized from the model checkpoint at nateraw/vit-base-beans and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 840/840 [00:47<00:00, 17.79it/s]


Evaluating model: Swin


100%|██████████| 840/840 [01:38<00:00,  8.55it/s]


Evaluating model: ConvNeXT


...

Evaluating model: ResNet152


/home/utkarsh/miniconda3/envs/pytorch_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/utkarsh/miniconda3/envs/pytorch_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 840/840 [01:29<00:00,  9.41it/s]


Evaluating model: DEiT


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTModel were not initialized from the model checkpoint at facebook/deit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 840/840 [00:39<00:00, 21.20it/s]


All Models Evaluation Results:
       Model  Top-1 Accuracy (%)  Top-3 Accuracy (%)  Top-5 Accuracy (%)
0        ViT           99.642857           99.880952           99.880952
1       Swin           99.880952          100.000000          100.000000
2   ConvNeXT          100.000000          100.000000          100.000000
3     RegNet           99.880952          100.000000          100.000000
4   ResNet50          100.000000          100.000000          100.000000
5  ResNet101          100.000000          100.000000          100.000000
6  ResNet152           99.880952           99.880952          100.000000
7       DEiT          100.000000          100.000000          100.000000
